# Build a DF relating item category name and shop name to shop ids and item ids

In [1]:
TARGET_ENCODING_NPERM = 8
KFOLD_K = 5
KFOLD_ALPHA=0

### Load packages

In [2]:
import pandas as pd
import pickle
import numpy as np
import re
import gc
from sklearn import preprocessing
from sklearn import model_selection

### Read in Data

In [3]:
df = pd.read_csv('../gen_data/train2.csv')

y_train_list = pickle.load(open('../gen_data/y_train--features1.ipynb--.pickle','rb'))
x_train_list = pickle.load(open('../gen_data/x_train--features3.ipynb--.pickle','rb'))
x_test_list = pickle.load(open('../gen_data/x_test--features3.ipynb--.pickle','rb'))
sales = pd.read_csv('../original_data/sales_train.csv.gz')

In [4]:
X = x_train_list[1].sample(10000)

In [5]:
first_sale_date = sales.groupby('item_id').date_block_num.min()
sales['first_sale_date_block'] = sales.item_id.map(first_sale_date)

X = X.merge(sales[['item_id','first_sale_date_block']],on='item_id',how='left')

In [6]:
X.head()

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,item_cnt_month_back_12,sum_item_cat_sales_back_12,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,first_sale_date_block
0,31,24,10930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,aug,31,229.0,0.0,NaN,0.0,0
1,31,24,10930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,aug,31,229.0,0.0,NaN,0.0,0
2,31,24,10930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,aug,31,229.0,0.0,NaN,0.0,0
3,31,24,10930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,aug,31,229.0,0.0,NaN,0.0,0
4,31,24,10930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,aug,31,229.0,0.0,NaN,0.0,0


In [7]:
d = pd.DataFrame({'A':[1,34,54,1,67],'B':[2,5,4,7,8]})
d

,A,B
0,1,2
1,34,5
2,54,4
3,1,7
4,67,8


In [8]:
d['C'] = (d.B > d.A)*1

In [9]:
d

,A,B,C
0,1,2,1
1,34,5,0
2,54,4,0
3,1,7,1
4,67,8,0


In [22]:
#start with sales data, figure out when each item was sold, 
    #merge this with train/test then transform it to 1/0
#fillna with new item?
def new_item_var(df,sales):
    first_sale_date = sales.groupby('item_id').date_block_num.min()
    sales['first_sale_date_block'] = sales.item_id.map(first_sale_date)
    sales = sales[['item_id','first_sale_date_block']]
    sales = sales.drop_duplicates()
    df = df.merge(sales,on='item_id',how='left')
    
    #handle nans for test data (which sales wont pick up)
    df.first_sale_date_block.fillna(999,inplace=True)    
    df['new_item'] = (df.first_sale_date_block >= df.date_block_num)*1

    return df

In [11]:
np.nan >= 5

False

In [13]:
#out[out.first_sale_date_block.isna()][['date_block_num','item_id','first_sale_date_block','new_item']].apply(lambda x: type(x.values))

In [14]:
test_items = x_test_list[1].item_id.unique()
sales_items = sales.item_id.unique()
new_item1 = np.isin(test_items,sales_items)
1-np.mean(np.isin(test_items,sales_items))

0.07117647058823529

In [15]:
test_shops = x_test_list[1].shop_id.unique()
len(test_shops) * len(test_items)

214200

In [23]:
out = new_item_var(x_test_list[1],sales)
print(np.mean(out.first_sale_date_block.isna()))
print(np.mean(out.new_item))
out_small = out[['date_block_num','item_id','first_sale_date_block','new_item']]
out_small

0.0
0.0711764705882353


,date_block_num,item_id,first_sale_date_block,new_item
0,34,5037,20.0,0
1,34,5320,999.0,1
2,34,5233,27.0,0
3,34,5232,31.0,0
4,34,5268,999.0,1
5,34,5039,20.0,0
6,34,5041,32.0,0
7,34,5046,22.0,0
8,34,5319,22.0,0
9,34,5003,32.0,0


In [24]:
isnew = out_small.groupby('item_id').new_item.mean()
print(isnew.unique())
np.mean(isnew)

[0 1]


0.0711764705882353

In [25]:
#what proportion of new items make rows in test data
new_item1

array([ True, False,  True, ...,  True,  True,  True])

In [ ]:
df = x_train_list[0]
df['target'] = y_train_list[0]

In [ ]:
x_train_list[1][x_train_list[1].item_id==30].item_category_id

### Parse City data

In [5]:
#cities_encoder = preprocessing.LabelEncoder()
#cities_series = pd.Series([re.search('(.*?) ',n).group() for n in df.shop_name],index=df.index)
#cities_encoder.fit(cities_series)
#df['city'] = cities_encoder.transform(cities_series)
#del cities_series

cities_series = pd.Series([re.search('(.*?) ',n).group() for n in df.shop_name],index=df.index)
df['city'] = cities_series

### Parse item category data

In [6]:
#item_type_encoder = preprocessing.LabelEncoder()
#item_type_series = df.item_category_name.map(lambda x: re.split('-',x)[0])
#item_type_encoder.fit(item_type_series)
#df['item_type'] = item_type_encoder.transform(item_type_series)
#del item_type_series

item_type_series = df.item_category_name.map(lambda x: re.split('-',x)[0])
df['item_type'] = item_type_series

### parse more item_category data

In [7]:
#item_info_encoder = preprocessing.LabelEncoder()
#item_info_series = df.item_category_name.map(lambda x: '-'.join(re.split('-',x)[1:]) if len(re.split('-',x))>1 else 'NAN')
#item_info_encoder.fit(item_info_series)
#df['item_info'] = item_info_encoder.transform(item_info_series)
#del item_info_series

item_info_series = df.item_category_name.map(lambda x: '-'.join(re.split('-',x)[1:]) if len(re.split('-',x))>1 else 'NAN')
df['item_info'] = item_info_series

### Select relevant columns in df

In [8]:
df = df[['shop_id','item_id','item_category_id','city','item_type','item_info']].drop_duplicates()

### function to downcast data types to 32 bits

In [9]:
def downcast(df):
    float_cols = [col for col in df if df[col].dtype=='float64']
    int_cols = [col for col in df if df[col].dtype=='int64']

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int32)
    
    return(df)

### cleanup

In [10]:
df = downcast(df)
gc.collect()

7

### Merge data above into training and test sets

In [11]:
x_train_list = list(map(lambda x: x.merge(df,on=['item_id','item_category_id','shop_id'],how='left'),x_train_list))
x_test_list = list(map(lambda x: x.merge(df,on=['item_id','item_category_id','shop_id'],how='left'),x_test_list))

# check

In [42]:
x_train_list[1].city.unique()
#x_test_list[1].city.unique()
np.sum(cities_series=='!Якутск ')
df.city.unique()

array(['!Якутск ', 'Адыгея ', 'Балашиха ', 'Волжский ', 'Воронеж ',
       'Жуковский ', 'Интернет-магазин ', 'Казань ', 'Калуга ',
       'Коломна ', 'Красноярск ', 'Курск ', 'Москва ', 'Н.Новгород ',
       'Новосибирск ', 'Омск ', 'РостовНаДону ', 'СПб ', 'Самара ',
       'Сергиев ', 'Сургут ', 'Тюмень ', 'Уфа ', 'Химки ', 'Чехов ',
       'Ярославль ', 'Вологда ', 'Якутск ', 'Цифровой ', 'Выездная ',
       'Томск ', 'Мытищи '], dtype=object)

In [32]:
df[df.city=='!Якутск '].head()

,shop_id,item_id,item_category_id,city,item_type,item_info
0,0,32,40,!Якутск,Кино,DVD
1,0,33,37,!Якутск,Кино,Blu-Ray
2,0,35,40,!Якутск,Кино,DVD
3,0,43,40,!Якутск,Кино,DVD
4,0,51,57,!Якутск,Музыка,MP3


In [33]:
tc = x_train_list[1]
tc[(tc.shop_id==0)].head()

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,city,item_type,item_info


In [39]:
np.mean(tc.city.isna())


0.8565935377754784

In [45]:
#should be 0
print(np.sum(tc.city.isna()))
#should be 0
print(np.sum(tc.item_type.isna()))
#i would imagine should be small, sometimes info isnt parsed
print(np.mean(tc.item_info=='NAN'))

5503694
5503694
0.0006616245614461049


In [43]:
[print(x) for x in df.city.unique()]
print('\n\n')
[print(x) for x in df.item_type.unique()]
print('\n\n')
[print(x) for x in df.item_info.unique()]

!Якутск 
Адыгея 
Балашиха 
Волжский 
Воронеж 
Жуковский 
Интернет-магазин 
Казань 
Калуга 
Коломна 
Красноярск 
Курск 
Москва 
Н.Новгород 
Новосибирск 
Омск 
РостовНаДону 
СПб 
Самара 
Сергиев 
Сургут 
Тюмень 
Уфа 
Химки 
Чехов 
Ярославль 
Вологда 
Якутск 
Цифровой 
Выездная 
Томск 
Мытищи 



Кино 
Музыка 
Книги 
Программы 
Игры PC 
Подарки 
Игры 
Аксессуары 
Игровые консоли 
Карты оплаты 
Чистые носители (шпиль)
Чистые носители (штучные)
Элементы питания
Служебные
Доставка товара
PC 
Карты оплаты (Кино, Музыка, Игры)
Билеты (Цифра)
Игры MAC 
Игры Android 
Служебные 



 DVD
 Blu-Ray
 MP3
 Аудиокниги
 Аудиокниги 1С
 1С:Предприятие 8
 Методические материалы 1С
 Обучающие
 Стандартные издания
 Развитие
 CD локального производства
 Музыкальное видео
 Для дома и офиса
 PS3
 XBOX 360
 PSVita
 Дополнительные издания
 PSP
 Коллекционные издания
 Гаджеты, роботы, спорт
 CD фирменного производства
 PSN
 Аксессуары для игр
 Сувениры
 Live!
 Blu-Ray 3D
 Коллекционное
 Мягкие игрушки
 Сувениры (в

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### it is possible (should check) that all of the !Якутск transactions were early and thus not picked up, should still fix because datebackgen will change

In [53]:
np.sum((df.city=='!Якутск ')&df.date_block)

6123

In [52]:
tc[(tc.shop_id==50)].head()

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,city,item_type,item_info
271836,12,50,32,40.0,89.0,4358.0,4.0,26966.0,836.0,81.0,...,0.0,jan,31,134.691573,15.602202,149.0,0.917077,Тюмень,Кино,DVD
271837,12,50,33,37.0,42.0,4358.0,2.0,9906.0,231.0,0.0,...,82.0,jan,31,186.571429,16.449056,199.0,0.755580,Тюмень,Кино,Blu-Ray
271838,12,50,99,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,jan,31,124.167083,25.367019,NaN,0.000000,Тюмень,Кино,Blu-Ray
271839,12,50,482,0.0,88.0,4358.0,2.0,412.0,2.0,0.0,...,2.0,jan,31,2866.022727,435.543311,3300.0,0.996404,NaN,NaN,NaN
271840,12,50,485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,jan,31,250.203390,51.145216,NaN,0.000000,NaN,NaN,NaN


In [58]:
df[(df.item_id==30) & (df.shop_id==2)]

,shop_id,item_id,item_category_id,city,item_type,item_info
123159,2,30,40,Адыгея,Кино,DVD


In [61]:
tc.item_category_id.unique().min()

0.0

In [55]:
tc[tc.city.isna()][['item_id','item_category_id','shop_id']]

,item_id,item_category_id,shop_id
483,30,0.0,2
484,791,0.0,2
485,972,0.0,2
486,1201,0.0,2
487,1235,0.0,2
488,1285,0.0,2
489,1398,0.0,2
490,1408,0.0,2
491,1448,0.0,2
492,1459,0.0,2


# Target Encoding

Target encoding is based on historical information, available for both train and test datasets, as such, we needn't worry about regularisation specifically for target encodings.

In [12]:
def kfold_target_encode(x_train,y_train,x_test,cols,k=5,alpha=0):
    x_train['target'] = y_train
    globmean = x_train.target.mean()
#for test data
    for col in cols:
        agged_sum = x_train.groupby(col).target.sum()
        x_test[col+'_encoded'] = x_test[col].map(agged_sum)
        x_test[col+'_encoded'].fillna(globmean,inplace=True)
            
#for train data, need to regularise
    #ensure index is reset to avoid breaking later
    

    x_train.reset_index(drop=True,inplace=True)
    x_train[col+'_encoded']=np.nan
    for col in cols:
        print('Encoding '+col)
        folds = model_selection.KFold(n_splits=k,shuffle=True,random_state=0)
        for compute_ind , map_ind in folds.split(x_train):
            mean = x_train.iloc[compute_ind].groupby(col).target.mean()
            count = x_train.iloc[compute_ind].groupby(col).target.count()
            encoding = (mean*count + globmean*alpha)/(alpha+count)
            x_train.loc[map_ind,col+'_encoded'] = x_train.loc[map_ind,col].map(encoding) 
            x_train[col+'_encoded'].fillna(globmean,inplace=True)
    return x_train.drop('target',axis=1) , x_test

In [13]:
def perm_target_encode(x_train,y_train,x_test,cols,nperms):
    x_train['target'] = y_train
    print('top: '+str(x_train.shape))
#for test data
    for col in cols:
        agged_sum = x_train.groupby(col).target.sum()
        x_test[col+'_encoded'] = x_test[col].map(agged_sum)
        x_test[col+'_encoded'].fillna(0,inplace=True)
            
#for train data, need to regularise
    for col in cols:
        encoding_df = pd.DataFrame()
        for perm_num in range(0,nperms):
            print('\tComputing encoding on permutation '+str(perm_num)+' of '+str(nperms))
            #set random state to perm num for reproducibility
            #make this faster by just changing index?
            perm_x_train = x_train.sample(frac=1,random_state=perm_num)
            cumsum   = perm_x_train.groupby(col).target.cumsum() - perm_x_train.target
            cumcount = perm_x_train.groupby(col).target.cumcount()
            encoding = cumsum/cumcount
            
            encoding_df['perm_'+str(perm_num)+'_encoding'] = encoding
            
        encoding_df['colmeans'] = encoding_df.mean(axis=1)    
        x_train[col+'_encoded'] = encoding_df.colmeans
        x_train[col+'_encoded'].fillna(0,inplace=True)
        del encoding_df
        gc.collect()
    return x_train.drop('target',axis=1) , x_test

In [14]:
to_target_encode = ['shop_id','item_id','item_category_id','month','city','item_type','item_info']
to_target_encode=['shop_id']
jumbled_list = [kfold_target_encode(x_train,y_train,x_test,to_target_encode,KFOLD_K,alpha=KFOLD_ALPHA) for x_train , y_train, x_test in zip(x_train_list,y_train_list,x_test_list)]
x_train_list , x_test_list = list(map(list,zip(*jumbled_list)))

Encoding shop_id
Encoding shop_id


In [15]:
d = x_train_list[1]
d['target'] = y_train_list[1]

In [16]:
d[d.shop_id==50].shop_id_encoded.unique()

array([0.28727492, 0.28723065, 0.28725757, 0.28724433, 0.28727477])

In [26]:
x_train_list[0]['shop_id_encoded']

0          0.287324
1          0.287353
2          0.287350
3          0.287352
4          0.287353
5          0.287324
6          0.287353
7          0.287353
8          0.287355
9          0.287355
10         0.287355
11         0.287353
12         0.287324
13         0.287324
14         0.287324
15         0.287353
16         0.287324
17         0.287350
18         0.287352
19         0.287350
20         0.287352
21         0.287352
22         0.287324
23         0.287324
24         0.287324
25         0.287353
26         0.287352
27         0.287324
28         0.287352
29         0.287324
             ...   
6186892    0.287815
6186893    0.287811
6186894    0.287815
6186895    0.287819
6186896    0.287803
6186897    0.287815
6186898    0.287803
6186899    0.287811
6186900    0.287798
6186901    0.287803
6186902    0.287798
6186903    0.287798
6186904    0.287811
6186905    0.287819
6186906    0.287819
6186907    0.287819
6186908    0.287798
6186909    0.287819
6186910    0.287819


In [17]:
d.groupby('shop_id').target.mean()

shop_id
2     135451
3     135451
4     135451
5     135451
6     135451
7     135451
9      17223
10    129398
11      6053
12    135451
13     54941
14    135451
15    135451
16    135451
17     86412
18    135451
19    135451
20     11747
21    135451
22    135451
24    135451
25    135451
26    135451
27    124930
28    135451
29    109306
30     92465
31    135451
33     55445
34     93484
35    135451
36      5413
37    135451
38    135451
39    120766
40     71727
41    135451
42    135451
43     86412
44    135451
45    135451
46    135451
47    135451
48    113624
49    135451
50    135451
51    130038
52    135451
53    135451
54    104023
55    135451
56    135451
57    135451
58    135451
59    135451
Name: target, dtype: int64

### Save

In [18]:
pickle.dump(x_train_list,open('../gen_data/x_train--features4.ipynb--.pickle','wb'))
pickle.dump(x_test_list,open('../gen_data/x_test--features4.ipynb--.pickle','wb'))

In [19]:
x_train_list[0].head()

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,city,item_type,item_info,shop_id_encoded
0,12,2,32,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,jan,31,134.691573,15.602202,NaN,0.000000,1.0,11.0,6.0,0.287324
1,12,2,33,37.0,42.0,1255.0,1.0,9906.0,55.0,15.0,...,jan,31,186.571429,16.449056,199.0,0.755580,1.0,11.0,1.0,0.287353
2,12,2,99,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,jan,31,124.167083,25.367019,NaN,0.000000,1.0,11.0,1.0,0.287350
3,12,2,482,73.0,88.0,1255.0,1.0,412.0,3.0,85.0,...,jan,31,2866.022727,435.543311,3300.0,0.996404,1.0,15.0,0.0,0.287352
4,12,2,485,73.0,111.0,1255.0,1.0,412.0,3.0,0.0,...,jan,31,250.203390,51.145216,300.0,0.973632,1.0,15.0,0.0,0.287353


In [20]:
#to_target_encode = ['shop_id','item_id','item_category_id','month','city','item_type','item_info']
#jumbled_list = [perm_target_encode(x_train,y_train,x_test,to_target_encode,TARGET_ENCODING_NPERM) for x_train , y_train, x_test in zip(x_train_list,y_train_list,x_test_list)]
#x_train_list , x_test_list = list(map(list,zip(*jumbled_list)))

In [21]:
# need to add bool: item never sold before
                    #shop never sold before
# and int: num item sold
# num shop sold
# FOR EACH MONTH

#this is target encoding filling nans with 0 except missing early data

#doing this early in pipeline would be easy

#going back for all time might be a bad idea, the nature of the data would change thru time, (summingup)
#instaed just use dateback range

In [22]:
#create variables to quantify how much we know about a data point
#For each shop_id and item_id create sum of recent history(total sales for dateback_range)
#create bool if 0 indicating new shop or item
def generate_recent_history_summary(df):
    df_items = df.filter(regex='sum_item_sales_back_\d+$')
    df['historical_item_sales'] = df_items.sum(axis=1)
    df['new_item'] = (df.historical_item_sales<0.5)*1
    
    df_shops = df.filter(regex='sum_shop_sales_back_\d+$')
    df['historical_shop_sales'] = df_shops.sum(axis=1)
    df['new_shop'] = (df.historical_shop_sales<0.5)*1
    
    df_item_cats = df.filter(regex='sum_item_cat_sales_back_\d+$')
    df['historical_item_cat_sales'] = df_item_cats.sum(axis=1)
    df['new_item_cat'] = (df.historical_item_cat_sales<0.5)*1
    return df
    

In [23]:
x_train_list = list(map(generate_recent_history_summary,x_train_list))
x_test_list = list(map(generate_recent_history_summary,x_test_list))

In [24]:
x_train_list[0][['item_id','historical_item_sales']]

,item_id,historical_item_sales
0,32,0.0
1,33,118.0
2,99,0.0
3,482,1097.0
4,485,818.0
5,804,144.0
6,839,307.0
7,1007,463.0
8,1406,16.0
9,1407,726.0


In [25]:
df = x_train_list[0][['date_block_num','item_id','item_id_encoded','historical_item_sales']].copy()
df.columns = ['date_block_num','item_id','ORIGINAL_item_id_encoded','historical_item_sales']
df.head()

KeyError: "['item_id_encoded'] not in index"

In [ ]:
#work on this
#this isnt working properly, drop it, I have almost the same information from elsewhere
def old_target_encode(df,target,cols,fillna=0):
    maxmonth = df.date_block_num.max()
    print(maxmonth)
    df['target'] = target
    for col in cols:
        print('Encoding variable: '+col)
        #dont use current data, that way we can do this for both train and test sets
        agged_targ = df[df.date_block_num<maxmonth].groupby(col).target.sum()
        agged_targ = df[(df.date_block_num<maxmonth) & (df.date_block_num>=(maxmonth-12))].groupby(col).target.sum()
        #agged_targ = df[df.date_block_num>20].groupby(col).target.sum()
        
        
        
        df[col+'_encoded'] = df[col].map(agged_targ)
        if fillna=='mean':
            df[col].fillna(df(col).mean(),inplace=True)
        elif fillna=='median':
            df[col].fillna(df(col).median(),inplace=True)
        else:
            df[col].fillna(0,inplace=True)
    return df.drop('target',axis=1)

### Im counting back the months from max month, but historical is rolling for each date, historical is bad becuase it double counts, or does it?

In [ ]:
#targs = y_train_list[0]
#target_encode(df,targs,['item_id'])
#g = df[(df.date_block_num<32) & (df.date_block_num>=20)].groupby('item_id').target.sum()
#df['item_id_encoded_manual'] = df.item_id.map(g)
#df.head()

In [ ]:
21 ... 31

In [ ]:
df = x_train_list[1]
targs = y_train_list[1]
df['target'] = targs
df_small = df[['item_id','item_id_encoded','target','historical_item_sales']]

In [ ]:
g = df.groupby('item_id').target.sum()
g.head(40)

In [ ]:
df_small['item_id_encoded_manual'] = df.item_id.map(g)

In [ ]:
df_small.head()

In [ ]:
df_small.head(15)

In [ ]:
df[(df.item_id==32)].target.sum()

In [ ]:
#df[['item_id','item_idencoded']].sort_values('item_id').head(1000)

In [ ]:
df100 = df.sample(100)
df100